<a href="https://colab.research.google.com/github/Ajay7545/EEGClassification/blob/master/model2lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from random import shuffle

import pandas as pd
import tensorflow as tf
import io
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Input, Dense, core,TimeDistributed,AveragePooling1D

from tensorflow.python.keras.layers import LSTM, Dense, Dropout, Activation
from sklearn.preprocessing import minmax_scale
from sklearn import preprocessing

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
from pandas import read_csv
from numpy import mean
from numpy import std
from numpy import delete
from numpy import savetxt


W0626 11:52:23.926454 140609155762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0626 11:52:24.995739 140609155762048 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0626 11:52:26.871931 140609155762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0626 11:52:26.873590 140609155762048 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/d

In [0]:
df1.shape

(14979, 15)

In [0]:
df=pd.read_csv('drive/My Drive/COLAB Files/EEG Eye State.csv')
df.columns = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O']
df.head(2)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
0,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
1,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0


In [0]:
df.shape

(14979, 15)

In [0]:
y=df['O']
print(len(y))
print(y.shape)

14979
(14979,)


In [0]:
df.drop(['O'], axis = 1, inplace = True)
df.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N
0,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10
1,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23
2,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41
3,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46
4,4321.03,4004.62,4284.10,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74


In [0]:
#inputs = datalist[:, :-1]
#output = datalist[:, -1]

In [0]:
num_runs = 10
num_runs = num_runs + 1

train_loss_list = []
train_accuracy_list = []

val_loss_list = []
val_accuracy_list = []


In [0]:
import tensorflow as tf

In [0]:
Train_inputs, Test_inputs, train_output, test_output = train_test_split(df,y, test_size=0.20, shuffle=True, random_state=1)

In [0]:
std_scale = preprocessing.StandardScaler().fit(Train_inputs)
train_inputs = std_scale.transform(Train_inputs)
test_inputs = std_scale.transform(Test_inputs)

In [0]:

trintf=np.asarray(train_inputs)
trouttf=np.asarray(train_output)
tsintf=np.asarray(test_inputs)
tsouttf=np.asarray(test_output)

#train_inputs_tf = tf.convert_to_tensor(train_inputs, np.float32)
#train_output_tf = tf.convert_to_tensor(train_output, np.float32)




In [0]:
print(trintf.shape)
print(11983*14)

(11983, 14)
167762


In [0]:

trintf=trintf.reshape(11983,1,14)
trouttf=trouttf.reshape(11983,1,1)
tsintf=tsintf.reshape(2996,1,14)
tsouttf=tsouttf.reshape(2996,1,1)

In [0]:
batch_size = 1
training_split = 0.25
num_fields = 14

In [0]:
model = Sequential()
model.add(LSTM(100,return_sequences=True,implementation=2))
model.add(Dropout(0.5))
model.add(LSTM(100,return_sequences=True,implementation=2))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])


W0626 11:53:02.094851 140609155762048 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
history = model.fit(trintf, trouttf, epochs=700, batch_size= 128,verbose=1,validation_data=(tsintf, tsouttf))


Train on 11983 samples, validate on 2996 samples
Epoch 1/700
11983/11983 [==============================] - 2s 164us/sample - loss: 0.0967 - acc: 0.8686 - val_loss: 0.0872 - val_acc: 0.8858
Epoch 2/700
11983/11983 [==============================] - 2s 171us/sample - loss: 0.0968 - acc: 0.8692 - val_loss: 0.0869 - val_acc: 0.8845
Epoch 3/700
11983/11983 [==============================] - 2s 159us/sample - loss: 0.0963 - acc: 0.8691 - val_loss: 0.0866 - val_acc: 0.8858
Epoch 4/700
11983/11983 [==============================] - 2s 162us/sample - loss: 0.0950 - acc: 0.8690 - val_loss: 0.0863 - val_acc: 0.8818
Epoch 5/700
11983/11983 [==============================] - 2s 166us/sample - loss: 0.0963 - acc: 0.8678 - val_loss: 0.0854 - val_acc: 0.8855
Epoch 6/700
11983/11983 [==============================] - 2s 171us/sample - loss: 0.0951 - acc: 0.8707 - val_loss: 0.0853 - val_acc: 0.8875
Epoch 7/700
11983/11983 [==============================] - 2s 166us/sample - loss: 0.0940 - acc: 0.8716 -

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']

train_loss_list.append(loss[-1])
val_loss_list.append(val_loss[-1])

acc = history.history['acc']
val_acc = history.history['val_acc']
epoch = range(1,len(loss)+1)

train_accuracy_list.append(acc[-1])
val_accuracy_list.append(val_acc[-1])

In [0]:
#print("Number of run " + str(run))
print("Train loss " + loss[-1].astype(str))
print("Validation loss " + val_loss[-1].astype(str))
print("Train accuracy " + acc[-1].astype(str))
print("Validation accuracy " + val_acc[-1].astype(str))

Train loss 0.03960783818916517
Validation loss 0.03564444231553294
Train accuracy 0.94876075
Validation accuracy 0.95293725


In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [0]:
a=2
a

2

In [0]:
x_val = [x for x in range(1, 51)]
plt.figure()
plt.plot(x_val, acc,'bo', label= 'Training accuracy')
plt.plot(x_val, val_acc,'b', label= 'Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(x_val, loss,'bo', label= 'Training loss')
plt.plot(x_val, val_loss,'b', label= 'Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()


In [0]:
plt.figure()
plt.plot(x_values,train_accuracy_list,'bo',label= 'Training accuracy per run')
plt.plot(x_values,val_accuracy_list,'b',label= 'Validation accuracy per run')
plt.title('Training and Validation accuracy per run')
plt.legend()
plt.show()

plt.figure()
plt.plot(x_values,train_loss_list,'bo',label= 'Training loss per run')
plt.plot(x_values,val_loss_list,'b',label= 'Validation loss per run')
plt.title('Training and Validation loss per run')
plt.legend()
plt.show()

average_train_loss = sum(train_loss_list, 0.0)/len(train_loss_list)
average_train_accuracy = sum(train_accuracy_list, 0.0)/len(train_accuracy_list)
print("Average train loss " + average_train_loss.astype(str))
print("Average train accuracy " + average_train_accuracy.astype(str))

average_val_loss = sum(val_loss_list, 0.0)/len(val_loss_list)
average_val_accuracy = sum(val_accuracy_list, 0.0)/len(val_accuracy_list)
print("Average val loss " + average_val_loss.astype(str))
print("Average val accuracy " + average_val_accuracy.astype(str))

In [0]:
a=2
a

2